In [1]:
import pandas as pd
import sqlite3
import os
import pickle

path=r"C:\Users\berid\python\yts mx project"

In [2]:
conn=sqlite3.connect(os.path.join(path,'scraped_data','yts.db'))

In [3]:
df=pd.read_sql_query("SELECT * FROM movie_details",conn)

In [4]:
df=df.replace('None',None)

In [5]:
year=df['YEAR'].apply(lambda x:x.split(' ')[0] if isinstance(x,str) else None)
language=df['YEAR'].apply(lambda x:x.split('[')[-1] .split(']')[0] if isinstance(x,str) and '[' in str(x) else 'ENGLISH')

df['YEAR']=year
df.insert(df.columns.get_loc('YEAR')+1,'LANGUAGE',language)


In [6]:
df['GENRES']=df['GENRES'].apply(lambda x:'|'.join([i.strip() for i in x.split('/')]) if isinstance(x,str) else None)

In [7]:
rating=df['IMDB'].apply(lambda x:x.split('/10')[0].replace('\n','').strip() if isinstance(x,str) else None).apply(pd.to_numeric)

rating_count=df['IMDB'].apply(lambda x:x.split('·')[-1].replace('\n','') if isinstance(x,str) else None)
rating_count=rating_count.apply(lambda x:
                   float(x.split('K')[0].strip()) * 1000 if isinstance(x, str) and 'K' in str(x) else
                   float(x.strip()) if isinstance(x, str) and x.strip().isdigit() else None
                   ).apply(pd.to_numeric, errors='coerce')


df.insert(df.columns.get_loc('IMDB')+1,'IMDB_RATING',rating)
df.insert(df.columns.get_loc('IMDB')+2,'IMDB_RATING_COUNT',rating_count)

df=df.drop(columns='IMDB')

In [8]:
df['CAST']=df['CAST'].apply(lambda x:None if x=='[]' else x)

In [9]:
df['PLOT']=df['PLOT'].apply(lambda x: x.split('\nPlot summary\n')[-1].split('\n\n Uploaded by')[0] if isinstance(x,str) else None)

In [10]:
df['TAGS']=df['TAGS'].apply(lambda x:'|'.join([i.strip() for i in x.split('\n')]) if isinstance(x,str) else None)

In [11]:
df=df[df['IMDB_RATING_COUNT']>=20000]

df=df.reset_index(drop=True)

In [12]:
#additional filtering and sorting
df=df[df['GENRES'].str.contains('Documentary',case=False)==False]
df=df.sort_values('IMDB_RATING_COUNT',ascending=False)
df=df.reset_index(drop=True)

### create tags from plot

In [13]:
import nltk
from nltk.util import ngrams
from collections import Counter
import contractions

from nltk.sentiment import SentimentIntensityAnalyzer
sia=SentimentIntensityAnalyzer()

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

from nltk.corpus import stopwords
stopwords=stopwords.words('english')
stopwords_extended = stopwords+['a', 'the', 'but', 'and', 's', '.', ',', "'s", 'i', 'in', 'to',':',';',"'",'(',')','–']


In [14]:
def get_bigrams(text):
    text = contractions.fix(text)
    plot_tokenized = nltk.word_tokenize(text)
    tokens_lowered = [t.lower().strip() for t in plot_tokenized]
    removed_stopwords = [t for t in tokens_lowered if t not in stopwords_extended]
    return list(ngrams(removed_stopwords, 2))

# Extract bigrams for all plots and count them
all_bigrams = []
df['PLOT'].apply(lambda plot: all_bigrams.extend(get_bigrams(plot)))
df['BIGRAMS']=df['PLOT'].apply(lambda plot:get_bigrams(plot))

# Count the most common bigrams
bigram_counts = Counter(all_bigrams)
most_common_bigrams = bigram_counts.most_common(10000)
most_common_bigrams=[i[0] for i in most_common_bigrams]

In [15]:
bigrams_series=[]
for i,bigrams_list in enumerate(df['BIGRAMS'],start=1):
    new_bigrams_list=[]
    for bigram in bigrams_list:
        if bigram in most_common_bigrams:
            new_bigrams_list.append(bigram)
    bigrams_series.append(new_bigrams_list)
    
    print(f'{i}/{len(df['BIGRAMS'])}',end='\r')

In [16]:
df['BIGRAMS']=bigrams_series

In [17]:
bigram_texts_series=[]

for i,bigrams_list in enumerate(df['BIGRAMS'],start=1):
    bigram_texts_list=[]
    for bigram in bigrams_list:
        bigram_text=' '.join([w for w in bigram])
        bigram_texts_list.append(bigram_text)
    
    bigram_texts_list='|'.join(i for i in bigram_texts_list)
    bigram_texts_series.append(bigram_texts_list)

df['BIGRAMS_TAGS']=bigram_texts_series
df=df.drop(columns='BIGRAMS')

In [18]:
#save preprocessed df
df.to_csv(os.path.join(path,'scraped_data','cleaned_yts_movies_df.csv'),index=False)